<a href="https://colab.research.google.com/github/Shubhpatel40/image_captioning/blob/main/image_captioning_using_custom_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adityajn105/flickr8k")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flickr8k


In [2]:
!apt install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libcudnn8
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 446 MB of archives.
After this operation, 1,140 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  libcudnn8 8.6.0.163-1+cuda11.8 [446 MB]
Fetched 446 MB in 6s (80.0 MB/s)
Selecting previously unselected package libcudnn8.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.6.0.163-1+cuda11.8_amd64.deb ...
Unpacking libcudnn8 (8.6.0.163-1+cuda11.8) ...
Setting up libcudnn8 (8.6.0.163-1+cuda11.8) ...


In [3]:
!pip uninstall -y tensorflow estimator keras

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0


In [4]:
!pip install -U tensorflow_text tensorflow tensorflow_datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow_text
    Found existing installation: tensorflow-text 2.18.1
    Uninstalling tensorflow-text-2.18.1:
      Successfully uninstalled tensorflow-text-2.18.1
ERROR: pip's dependency resolver does not currently take into account all the pac

In [5]:
!pip install einops

In [6]:
#@title
import concurrent.futures
import collections
import dataclasses
import hashlib
import itertools
import json
import math
import csv
import os
import pathlib
import random
import re
import string
import time
import urllib.request

import einops
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from PIL import Image
import requests
import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds
import sklearn

In [7]:
captions_file = path + "/captions.txt"

with open(captions_file, "r") as f:
    captions = f.read().splitlines()

In [8]:
full_dataset_dict = {}

for caption in captions[1:-1]:
    reader = csv.reader([caption])
    image, caption = tuple(next(reader))
    if image not in full_dataset_dict:
        full_dataset_dict[image] = []
    full_dataset_dict[image].append(caption)

### ***Load the model***

In [9]:
IMAGE_SHAPE = (224, 224, 3)
restnet = tf.keras.applications.ResNet50V2(
    input_shape=IMAGE_SHAPE,
    include_top=False)
restnet.trainable=False

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


***function to load the model and resize for the model***

In [10]:
def load_image(image_path):
    full_image_path = f"{path}/Images/{image_path}"
    img = tf.io.read_file(full_image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SHAPE[:-1])
    return img

In [11]:
print(tf.newaxis)

None


In [12]:
os.path.exists(f"{path}/Images/1000268201_693b08cb0e.jpg")

True

In [13]:
test_img_batch = load_image(list(full_dataset_dict.keys())[0])[tf.newaxis, :]

print(test_img_batch.shape)
print(restnet(test_img_batch).shape)

(1, 224, 224, 3)
(1, 7, 7, 2048)


## ***Preprocess Captions***

***Function for the remove punctuation, lower case, and add the special token before and after the captions.***

In [14]:
from typing import List
class preprocess_text:
   def __init__(self):
      pass

   def standardize(self, s:List):
      s = tf.constant(s, dtype=tf.string)
      s = tf.strings.lower(s)
      s = tf.strings.regex_replace(s, f'[{re.escape(string.punctuation)}]', '')
      byte_arr = s.numpy()

      # Decode each element safely
      decoded = [
          b.decode('utf-8') if isinstance(b, (bytes, bytearray)) else str(b)
          for b in byte_arr
      ]

      return [f"{t} [END]" for t in decoded]

   def make_vocabulary_dict(self, full_dataset_dict):
      vocabulary_size = 1
      vocab_dict = {'[START]': 0}
      for image in full_dataset_dict:
          for caption in full_dataset_dict[image]:
              for word in caption.split():
                  if word not in vocab_dict:
                      vocab_dict[word] = vocabulary_size
                      vocabulary_size += 1
                  else:
                      continue

      return vocab_dict

   def index_to_word(self, vocab_dict):
      index_to_word = {index: word for word, index in vocab_dict.items()}
      return index_to_word

   def word_to_index(self, s):
      return [vocab_dict[word] for word in s.split()]

In [15]:
class tokens_to_tensor:
    def __init__(self, vocab_dict, vocabulary_size, max_len=50):
      self.vocab_dict = vocab_dict
      self.vocabulary_size = vocabulary_size
      self.max_len = max_len
      self.preprocess_text = preprocess_text()

    def preprocessing(self, caption_batch):
      all_captions_tokenIDs = []

      for s_tokenID in caption_batch:
        ## split the caption
        s_tokenID = []

        ## find the token ID for the each token from vocab_dict.
        # for word in caption:
        #   if word in self.vocab_dict:
        #     s_tokenID.append(self.vocab_dict[word])

        #   else:
        #     s_tokenID.append(0)

        ## padding the length
        for _ in range(0, self.max_len-len(s_tokenID)):
          s_tokenID.append(0)

        all_captions_tokenIDs.append(s_tokenID)

      tgt_ids = torch.tensor(all_captions_tokenIDs, dtype=torch.long)
      return tgt_ids

In [16]:
# test_preprocess = preprocess_text(vocab_dict, vocabulary_size)
# standardize(["a cat on the sat, and i would like to tell with her."])

In [17]:
# temp_captions = list(standardize(full_dataset_dict["2083434441_a93bc6306b.jpg"]))
# print(temp_captions)

In [18]:
preprocessing = preprocess_text()

In [19]:
for image in full_dataset_dict:
    full_dataset_dict[image] = preprocessing.standardize(full_dataset_dict[image])

In [20]:
full_dataset_dict["2083434441_a93bc6306b.jpg"]

['an elderly woman is riding a bicycle in the city as a yellow taxi is about to pass by  [END]',
 'an elderly woman rides a bicycle along a city street  [END]',
 'an older woman with blond hair rides a bicycle down the street  [END]',
 'a woman in a grey overcoat rides her bicycle along a street  [END]',
 'older woman wearing glassses riding a bicycle with a shopping bag on the handle  yellow car is in the background  [END]']

Make the vocabulary size

In [21]:
vocab_dict = preprocessing.make_vocabulary_dict(full_dataset_dict)

***Convert the dataset into the index***

In [22]:
full_dataset = {}
for image in full_dataset_dict:
    for caption in full_dataset_dict[image]:
        full_dataset[caption] = image

In [23]:
print(f'length of the full dataset each caption with image: {len(list(full_dataset.keys()))}')

length of the full dataset each caption with image: 40168


***Make the index to word ***

In [24]:
index_to_word = preprocessing.index_to_word(vocab_dict)

### ***Make the dataframe from the dictionary***

In [25]:
df = pd.DataFrame.from_dict(full_dataset, orient='index', columns=['image'])
df = df.reset_index().rename(columns={'index': 'caption'})
df

,caption,image
0,a child in a pink dress is climbing up a set o...,1000268201_693b08cb0e.jpg
1,a girl going into a wooden building [END],1000268201_693b08cb0e.jpg
2,a little girl climbing into a wooden playhouse...,1000268201_693b08cb0e.jpg
3,a little girl climbing the stairs to her playh...,1000268201_693b08cb0e.jpg
4,a little girl in a pink dress going into a woo...,1000268201_693b08cb0e.jpg
...,...,...
40163,woman writing on a pad in room with gold deco...,997338199_7343367d7f.jpg
40164,a man in a pink shirt climbs a rock face [END],997722733_0cb5439472.jpg
40165,a man is rock climbing high in the air [END],997722733_0cb5439472.jpg
40166,a person in a red shirt climbing up a rock fac...,997722733_0cb5439472.jpg


In [26]:
df['caption'] = df['caption'].apply(lambda x: x.replace("  ", " "))
df['caption'] = df['caption'].apply(lambda x: preprocessing.word_to_index(x))

In [27]:
df

,caption,image
0,"[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 1...",1000268201_693b08cb0e.jpg
1,"[1, 16, 17, 18, 1, 19, 20, 15]",1000268201_693b08cb0e.jpg
2,"[1, 21, 16, 7, 18, 1, 19, 22, 15]",1000268201_693b08cb0e.jpg
3,"[1, 21, 16, 7, 23, 11, 24, 25, 22, 15]",1000268201_693b08cb0e.jpg
4,"[1, 21, 16, 3, 1, 4, 5, 17, 18, 1, 19, 26, 15]",1000268201_693b08cb0e.jpg
...,...,...
40163,"[234, 1258, 38, 1, 4025, 3, 561, 35, 1202, 306...",997338199_7343367d7f.jpg
40164,"[1, 75, 3, 1, 4, 161, 112, 1, 164, 202, 15]",997722733_0cb5439472.jpg
40165,"[1, 75, 6, 164, 7, 488, 3, 23, 227, 15]",997722733_0cb5439472.jpg
40166,"[1, 187, 3, 1, 110, 161, 7, 8, 1, 164, 202, 54...",997722733_0cb5439472.jpg


In [28]:
# # Convert dict keys and values to lists
# keys = list(full_dataset.keys())
# values = list(full_dataset.values())

# # Split keys and corresponding values together
# keys_train, keys_test, vals_train, vals_test = sklearn.model_selection.train_test_split(
#     keys, values,
#     test_size=0.2,
#     random_state=42,  # for reproducibility
#     shuffle=True
# )

# # Reconstruct train/test dicts
# train_data = dict(zip(keys_train, vals_train))
# test_data  = dict(zip(keys_test, vals_test))

In [29]:
# print(f"Train data lengh: {len(list(train_data.keys()))}")
# print(f"Test data length: {len(list(test_data.keys()))}")

***Make the dataframe from the dictionary***

In [30]:
# df_train = pd.DataFrame.from_dict(train_data, orient='index', columns=['value'])
# df_train = df_train.reset_index().rename(columns={'index': 'key'})
# df_train

In [31]:
# df_test = pd.DataFrame.from_dict(test_data, orient='index', columns=['value'])
# df_test = df_test.reset_index().rename(columns={'index': 'key'})
# df_test

***Feature extractor that extract the feature from the image***

In [32]:
def feature_extractor(img):
    resize_img = load_image(img)[tf.newaxis, :]
    img_feature = restnet(resize_img)
    return img_feature

## ***Transformer Embedding part of token***

In [33]:
len(list(vocab_dict.keys()))

8830

In [34]:
vocabulary_size = len(list(vocab_dict.keys()))
vocabulary_size

8830

In [35]:
import math
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 50):
        super().__init__()
        # Create a long enough P matrix once
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len,1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))  # (d_model/2,)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape (batch_size, seq_len, d_model)
        Returns:
            x + positional encoding, same shape
        """
        seq_len = x.size(1)
        # only take the first seq_len positions
        return x + self.pe[:, :seq_len]

class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size: int, max_length, depth: int):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, depth)
        self.pos_enc     = PositionalEncoding(depth, max_length)
        # this single layer can serve as demonstration; in practice you’d stack several of these
        # self.self_attn   = nn.MultiheadAttention(d_model, nhead)
        # self.ffn         = nn.Sequential(
        #     nn.Linear(d_model, 4*d_model),
        #     nn.ReLU(),
        #     nn.Linear(4*d_model, d_model)
        # )
        # self.layernorm1  = nn.LayerNorm(d_model)
        # self.layernorm2  = nn.LayerNorm(d_model)

    def forward(self, tgt: torch.Tensor) -> torch.Tensor:
        """
        Args:
            tgt: target token IDs, shape (batch_size, tgt_len)
        Returns:
            output embeddings, shape (batch_size, tgt_len, d_model)
        """
        bsz, tgt_len = tgt.size()

        # 1) Token embedding
        tok_emb = self.token_embed(tgt)                     # (bsz, tgt_len, d_model)

        # 2) Positional embedding
        x = self.pos_enc(tok_emb)                           # (bsz, tgt_len, d_model)

        return x

# Example usage
# if __name__ == "__main__":
#     B, L, V, D, H = 2, 10, 1000, 512, 8
#     decoder = EmbeddingLayer(vocab_size=V, max_length=50, depth=D)
#     test_tensor = convert_output_to_tensor(vocab_dict=vocab_dict, vocabulary_size=vocabulary_size)
#     tgt_ids = test_tensor.preprocessing([[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 12, 13, 14, 15], [1, 16, 17, 18, 1, 19, 20, 15]])
#     out = decoder(tgt_ids)                                # (B, L, D)
#     print(out)
#     print(out.shape)  # -> torch.Size([2, 10, 512])


In [36]:
class SeqEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, max_length, depth):
    super().__init__()
    self.pos_embedding = tf.keras.layers.Embedding(input_dim=max_length, output_dim=depth)

    self.token_embedding = tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=depth,
        mask_zero=True)

    self.add = tf.keras.layers.Add()

    self.max_length = max_length

  def __call__(self, seq):
    # seq = convert_into_index(seq)
    print(f"Calling Sequence embedding: {seq}")

    for _ in range(0, self.max_length-len(seq)):
        seq.append(0)

    seq = np.array(seq, dtype=np.int32)
    print(f"Sequence for embedding: {seq}")
    seq = self.token_embedding(seq) # (batch, seq, depth)

    print(f"After token embedding: {seq.shape}\n")

    token_emb = seq

    x = tf.range(tf.shape(seq)[0])  # (seq)
    x = x[tf.newaxis, :]  # (1, seq)
    print(x.shape)
    x = self.pos_embedding(x)  # (1, seq, depth)

    print(f"After positional embedding: {x.shape}\n")

    return self.add([seq, token_emb[np.newaxis, :, :]])

In [37]:
class CausalSelfAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    # Use Add instead of + so the keras mask propagates through.
    self.add = tf.keras.layers.Add()
    self.layernorm = tf.keras.layers.LayerNormalization()

  def __call__(self, x):
    attn = self.mha(query=x, value=x,
                    use_causal_mask=True)
    x = self.add([x, attn])
    return self.layernorm(x)

In [38]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self,**kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.add = tf.keras.layers.Add()
    self.layernorm = tf.keras.layers.LayerNormalization()

  def __call__(self, x, y, **kwargs):
    attn, attention_scores = self.mha(
             query=x, value=y,
             return_attention_scores=True)

    self.last_attention_scores = attention_scores

    x = self.add([x, attn])
    return self.layernorm(x)

In [39]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, units, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
        tf.keras.layers.Dense(units=2*units, activation='relu'),
        tf.keras.layers.Dense(units=units),
        tf.keras.layers.Dropout(rate=dropout_rate),
    ])

    self.layernorm = tf.keras.layers.LayerNormalization()

  def __call__(self, x):
    x = x + self.seq(x)
    return self.layernorm(x)


In [40]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, units, num_heads=1, dropout_rate=0.1):
    super().__init__()

    self.self_attention = CausalSelfAttention(num_heads=num_heads,
                                              key_dim=units,
                                              dropout=dropout_rate)
    self.cross_attention = CrossAttention(num_heads=num_heads,
                                          key_dim=units,
                                          dropout=dropout_rate)
    self.ff = FeedForward(units=units, dropout_rate=dropout_rate)


  def __call__(self, inputs, training=False):
    in_seq, out_seq = inputs

    print("Sucessfully entering into the decoder layer")
    # Text input
    out_seq = self.self_attention(out_seq)

    print(f"After completing self attention layer: {out_seq.shape}\n")

    out_seq = self.cross_attention(out_seq, in_seq)

    print(f'After completing cross attention layer: {out_seq}\n')

    self.last_attention_scores = self.cross_attention.last_attention_scores

    print(f"After completing last attention score: {self.last_attention_scores}\n")

    out_seq = self.ff(out_seq)

    print(f'After feed forward neural network: {out_seq}\n')

    return out_seq

In [41]:
#@title
class TokenOutput(tf.keras.layers.Layer):
  def __init__(self, banned_tokens=('', '[UNK]', '[START]'), **kwargs):
    super().__init__()

    self.dense = tf.keras.layers.Dense(
        units=vocabulary_size, activation='softmax', **kwargs)
    self.banned_tokens = banned_tokens

    self.bias = None

  def adapt(self, ds):
    counts = collections.Counter()

    for tokens in tqdm.tqdm(ds):
      counts.update(tokens)

    print(f"Counts in the Token Output: {counts}\n")

    counts_arr = np.zeros(shape=(vocabulary_size,))
    counts_arr[np.array(list(counts.keys()), dtype=np.int32)] = list(counts.values())

    counts_arr = counts_arr[:]
    for token in self.banned_tokens:
      counts_arr[vocab_dict[token]] = 0

    print(f"Counts array after removing banned tokens: {counts_arr}\n")
    print(f'Shape of the counts array: {counts_arr.shape}\n')

    total = counts_arr.sum()
    print(f"Total: {total}\n")
    p = counts_arr/total   ## get the empirical probability of each token. This is used to compute entropy and bias terms in probability space.
    print(f"Probability: {p}\n")
    p[counts_arr==0] = 1.0
    log_p = np.log(p)  # log(1) == 0

    print(f"log_p: {log_p}\n")

    entropy = -(log_p*p).sum()  ## Computes the marginal entropy. it describe how much uncertainty remains if you only knew the marginal token distribution.

    print()
    print(f"Uniform entropy: {np.log(vocabulary_size):0.2f}")
    print(f"Marginal entropy: {entropy:0.2f}")

    self.bias = log_p
    self.bias[counts_arr==0] = -1e9
    print(f"Bias: {self.bias}\n")

  def __call__(self, x):
    print(f"Successfully enter into the output layer: {x}")
    print(f"input shape of the output layer: {x.shape}")
    x = self.dense(x)
    # TODO(b/250038731): Fix this.
    # An Add layer doesn't work because of the different shapes.
    # This clears the mask, that's okay because it prevents keras from rescaling
    # the losses.
    return x

In [42]:
# convert_into_index('the dog is playing with the stick on the beach')

***Converting the caption into index***

In [43]:
# for index, caption in enumerate(df_train['key'], start=0):
#     caption = caption.replace("  ", " ")
#     caption = caption.replace("[END]", "")
#     df_train.at[index, 'key'] = convert_into_index(caption)

In [44]:
# for index, caption in enumerate(df_test['key'], start=0):
#     caption = caption.replace("  ", " ")
#     caption = caption.replace("[END]", "")
#     df_test.at[index, 'key'] = convert_into_index(caption)

In [45]:
# df_train

In [46]:
for key, value in vocab_dict.items():
    print(f"{key}: {value}")
    if value == 2:
      break

[START]: 0
a: 1
child: 2


In [47]:
output_layer = TokenOutput(banned_tokens=['[START]'])
# This might run a little faster if the dataset didn't also have to load the image data.
output_layer.adapt(df['caption'])

100%|██████████| 40168/40168 [00:00<00:00, 716739.57it/s]

Counts in the Token Output: Counter({1: 62683, 15: 40168, 3: 18876, 23: 18283, 38: 10707, 6: 9316, 29: 8840, 28: 8018, 35: 7756, 75: 7248, 10: 6707, 46: 5571, 40: 3927, 27: 3801, 146: 3575, 31: 3486, 234: 3400, 16: 3323, 24: 3172, 101: 3061, 44: 2913, 197: 2881, 141: 2752, 110: 2669, 73: 2629, 41: 2556, 12: 2430, 79: 2356, 162: 2268, 47: 2057, 118: 2035, 34: 1977, 123: 1976, 78: 1968, 386: 1821, 161: 1806, 142: 1787, 134: 1773, 21: 1766, 65: 1586, 2: 1543, 187: 1534, 174: 1465, 177: 1445, 166: 1402, 57: 1385, 346: 1373, 62: 1367, 91: 1324, 64: 1275, 8: 1259, 178: 1258, 80: 1247, 63: 1232, 127: 1225, 144: 1217, 196: 1216, 222: 1212, 25: 1178, 155: 1164, 438: 1151, 274: 1113, 18: 1070, 227: 1050, 136: 1041, 129: 1025, 223: 983, 230: 970, 185: 953, 192: 947, 45: 942, 210: 920, 131: 919, 646: 902, 128: 894, 138: 869, 294: 867, 548: 858, 435: 844, 74: 786, 37: 772, 374: 759, 86: 749, 140: 743, 164: 743, 50: 742, 94: 741, 4: 735, 1231: 731, 276: 725, 256: 693, 356: 691, 490: 690, 95: 680, 28

***Remove the ['START'] Token from the key in train and test dataset.***

In [48]:
# df_train['key'] = [a[1:] for a in df_train['key']]
# df_test['key'] = [a[1:] for a in df_test['key']]

In [49]:
# test_tensor = tokens_to_tensor(vocab_dict=vocab_dict, vocabulary_size=vocabulary_size)
#     tgt_ids = test_tensor.preprocessing([[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 12, 13, 14, 15], [1, 16, 17, 18, 1, 19, 20, 15]])
#     out = decoder(tgt_ids)

In [50]:
class Captioner(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, feature_extractor, output_layer, index_to_word, vocab_dict, vocabulary_size, num_layers=1,
               units=256, max_length=50, num_heads=1, dropout_rate=0.1):
    super().__init__()
    self.feature_extractor = feature_extractor
    # self.word_to_index = tf.keras.layers.StringLookup(
    #     mask_token="",
    #     vocabulary=tokenizer.get_vocabulary())
    self.index_to_word = index_to_word

    self.proj = tf.keras.layers.Dense(256)

    self.tensor = tokens_to_tensor(vocab_dict=vocab_dict, vocabulary_size=vocabulary_size)

    self.seq_embedding = EmbeddingLayer(
        vocab_size=vocabulary_size,
        depth=units,
        max_length=max_length)

    self.decoder_layers = [
        DecoderLayer(units, num_heads=num_heads, dropout_rate=dropout_rate)
        for n in range(num_layers)]

    self.output_layer = output_layer

In [51]:
  @Captioner.add_method
  def __call__(self, image, txt):
    if image.shape[-1] == 3:
      # Apply the feature-extractor, if you get an RGB image.
      image = self.feature_extractor(image)

    # Flatten the feature map
    # image = tf.reshape(image, [image.shape[0], -1])

    batch_size = tf.shape(image)[0]
    H, W, C = image.shape[1], image.shape[2], image.shape[3]

    # 1) flatten spatial dims
    image = tf.reshape(image, (batch_size, H*W, C))

    image = self.proj(image)

    print(f"Image shape: {image.shape}")

    ## convert the text into the tensor
    txt = self.tensor.preprocessing(txt)

    txt = self.seq_embedding(txt)

    txt = txt.detach().numpy()

    print(f"After embedding: {txt}")

    # Look at the image
    for dec_layer in self.decoder_layers:
      txt = dec_layer(inputs=(image, txt))

    print(f"After completing decoder layer: {txt}")
    txt = self.output_layer(txt)

    return txt

In [52]:
model = Captioner(feature_extractor=feature_extractor, output_layer=output_layer, index_to_word=index_to_word, vocab_dict=vocab_dict, vocabulary_size=vocabulary_size,
                  units=256, dropout_rate=0.5, num_layers=2, num_heads=2)

In [53]:
torch.tensor([[1],[2]], dtype=torch.long).detach().numpy()

array([[1],
       [2]])

In [54]:
# tf.concat([torch.tensor([1], dtype=torch.long), torch.tensor([2], dtype=torch.long)], axis=1)

In [ ]:
def sample_next_token(last_logits, temperature=1.0, top_k=None, top_p=None):
    # last_logits: (batch, vocab)
    # 1) temperature
    logits = last_logits / tf.cast(temperature, last_logits.dtype)

    # 2) top_k
    if top_k is not None and top_k > 0:
        values, _ = tf.math.top_k(logits, k=top_k)
        min_val   = values[:, -1, None]
        logits    = tf.where(logits < min_val,
                             tf.fill(tf.shape(logits), -1e9),
                             logits)

    # 3) top_p
    if top_p is not None and top_p < 1.0:
        sorted_logits = tf.sort(logits, direction="DESCENDING", axis=-1)
        sorted_probs  = tf.nn.softmax(sorted_logits, axis=-1)
        cum_probs     = tf.cumsum(sorted_probs, axis=-1)
        # find first index where cum_probs > top_p:
        cutoff_idx    = tf.reduce_min(
            tf.where(cum_probs > top_p,
                     tf.range(tf.shape(cum_probs)[-1]),
                     tf.fill(tf.shape(cum_probs), tf.shape(cum_probs)[-1]))
        )
        cutoff_val    = sorted_logits[:, cutoff_idx][:, None]
        logits        = tf.where(logits < cutoff_val,
                                 tf.fill(tf.shape(logits), -1e9),
                                 logits)

    # 4) sample
    return tf.random.categorical(logits, num_samples=1, dtype=tf.int32)

In [59]:
@Captioner.add_method
def simple_gen(self, image, temperature=1, top_p=0.9, top_k=30):
  initial = [[vocab_dict['[START]']]] # (batch, sequence)
  img_features = self.feature_extractor(image)

  tokens = initial # (batch, sequence)
  print(tokens)
  for n in range(50):
    preds = self(img_features, tokens).numpy()  # (batch, sequence, vocab)

    # print(f"Predictions: {preds.shape}")
    preds = preds[:,-1, :]  #(batch, vocab)
    print(f"Predictions: {preds}")
    print(f"Predictions shape: {preds.shape}")

    if temperature==0:
        next = np.argmax(preds, axis=-1)  # (batch, 1)
    else:
        logits = tf.convert_to_tensor(preds / temperature)
        print(f"Logits: {logits}")

        if top_k is not None and top_k > 0:
            values, _ = tf.math.top_k(logits, k=top_k)
            print(f"After getting top_k: {values} and shape is: {values.shape}")
            min_val   = values[:, -1, None]
            print(f"Minimum value: {min_val}")
            logits = tf.where(logits < min_val,
                                tf.fill(tf.shape(logits), -1e9),
                                logits)
            print(f"After filling the top k value as it is and otherwise fill as infinity: {logits}")

        if top_p is not None and top_p < 1.0:
            sorted_logits = tf.sort(logits, direction="DESCENDING", axis=-1)
            print(f"Sorted logits in descending: {sorted_logits}")
            sorted_probs = tf.nn.softmax(sorted_logits, axis=-1)
            print(f"After softmax layer: {sorted_probs}")
            cum_probs = tf.cumsum(sorted_probs, axis=-1)
            print(f"Cumulative probability: {cum_probs}")
            # find first index where cum_probs > top_p:
            cutoff_idx    = tf.reduce_min(
                tf.where(cum_probs > top_p,
                        tf.range(tf.shape(cum_probs)[-1]),
                        tf.fill(tf.shape(cum_probs), tf.shape(cum_probs)[-1]))
            )
            cutoff_val    = sorted_logits[:, cutoff_idx][:, None]
            logits        = tf.where(logits < cutoff_val,
                                    tf.fill(tf.shape(logits), -1e9),
                                    logits)

        next = tf.random.categorical(logits, num_samples=1)
        next = next.numpy().squeeze(-1)
        # next = tf.random.categorical(preds/temperature, num_samples=1)  # (batch, 1)
    print(f"Next: {next}")
    batch0_id = next[0]
    max_logit_val  = preds[0, batch0_id]
    print(f"Next token ID: {batch0_id} with logit {max_logit_val:.3f}")

    next_tokens = [index_to_word[i] for i in next]
    print(f"Next tokens: {next_tokens}")

    tokens = np.concatenate([tokens, next[:, None]], axis=1)
    # tokens = tf.concat([tokens, next], axis=1) # (batch, sequence)

    if next[0] == vocab_dict['[END]']:
      break

  print(tokens)
  words = [index_to_word[i] for i in tokens[0]]
  print(f"Words: {words}")
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  print(f"Result: {result}\n")
  return result.numpy().decode()

In [60]:
result = model.simple_gen("215214751_e913b6ff09.jpg", temperature=1.0)
print(result)

Streaming output truncated to the last 5000 lines.
After filling the top k value as it is and otherwise fill as infinity: [[-1.e+09 -1.e+09 -1.e+09 ... -1.e+09 -1.e+09 -1.e+09]]
Sorted logits in descending: [[ 2.6076782e-04  2.5839044e-04  2.4425582e-04 ... -1.0000000e+09
  -1.0000000e+09 -1.0000000e+09]]
After softmax layer: [[0.03333452 0.03333444 0.03333397 ... 0.         0.         0.        ]]
Cumulative probability: [[0.03333452 0.06666896 0.10000293 ... 1.         1.         1.        ]]
Next: [7557]
Next token ID: 7557 with logit 0.000
Next tokens: ['661']
Image shape: (1, 49, 256)
After embedding: [[[-0.7525832   1.1109288  -0.7345768  ...  2.4231927  -2.0381057
   -0.9003954 ]
  [ 0.08888775  0.65123105  0.06738496 ...  2.4231927  -2.0379982
   -0.9003954 ]
  [ 0.1567142  -0.3052181   0.22356755 ...  2.4231927  -2.037891
   -0.9003954 ]
  ...
  [-0.6290101  -0.8814067  -0.97753286 ...  2.423178   -2.033055
   -0.90040815]
  [-1.5208378  -0.52921563 -0.10177958 ...  2.4231772 

In [ ]:
for t in (0.0, 0.5, 1.0):
  result = model.simple_gen("215214751_e913b6ff09.jpg", temperature=t)
  print(result)

Streaming output truncated to the last 5000 lines.
   -1.3259632e+01 -1.3150390e+01]
  [-1.0000000e+09 -1.6074951e+00 -5.8079772e+00 ... -1.2946672e+01
   -1.3260161e+01 -1.3150123e+01]
  [-1.0000000e+09 -1.6066668e+00 -5.8099942e+00 ... -1.2947758e+01
   -1.3261295e+01 -1.3149625e+01]
  ...
  [-1.0000000e+09 -1.6053690e+00 -5.8106036e+00 ... -1.2945221e+01
   -1.3253238e+01 -1.3149155e+01]
  [-1.0000000e+09 -1.6071987e+00 -5.8093810e+00 ... -1.2942951e+01
   -1.3252829e+01 -1.3149305e+01]
  [-1.0000000e+09 -1.6083961e+00 -5.8084054e+00 ... -1.2941546e+01
   -1.3252921e+01 -1.3149692e+01]]]
Image shape: (1, 49, 256)
After embedding: [[[-0.5932783   1.6369275  -0.6040073  ...  0.68717843 -1.1401287
    1.5306277 ]
  [ 0.24819267  1.1772299   0.19795448 ...  0.68717843 -1.1400213
    1.5306277 ]
  [ 0.31601912  0.22078064  0.35413706 ...  0.68717843 -1.1399138
    1.5306277 ]
  ...
  [-0.46970516 -0.35540795 -0.8469634  ...  0.6871637  -1.1350781
    1.5306149 ]
  [-1.3615329  -0.0032168

TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.

In [ ]:
class temp_convert_output_to_tensor:
    def __init__(self, vocab_dict, vocabulary_size, max_len=50):
      self.vocab_dict = vocab_dict
      self.vocabulary_size = vocabulary_size
      self.max_len = max_len
      self.preprocess_text = preprocess_text()

    def preprocessing(self, caption_batch):
      all_captions_tokenIDs = []

      for s_tokenID in caption_batch:
    #     ## split the caption
    #     s_tokenID = []

    #     ## find the token ID for the each token from vocab_dict.
    #     for word in caption:
    #       if word in self.vocab_dict:
    #         s_tokenID.append(self.vocab_dict[word])

    #       else:
    #         s_tokenID.append(0)

        ## padding the length
        for _ in range(0, self.max_len-len(s_tokenID)):
          s_tokenID.append(0)

        all_captions_tokenIDs.append(s_tokenID)

      tgt_ids = torch.tensor(all_captions_tokenIDs, dtype=torch.long)
      return tgt_ids

tensor([[[-0.4055, -0.4849, -0.2018,  ...,  0.6381, -1.3429,  1.1430],
         [ 0.6739,  1.2321,  0.5574,  ...,  1.3237,  1.2483,  0.9730],
         [ 2.5945, -0.8670,  0.5196,  ...,  1.4123, -0.1160,  1.1735],
         ...,
         [ 0.4736, -1.4479,  2.0100,  ...,  2.1350,  2.7324,  0.1760],
         [-0.4182, -1.0957,  1.7640,  ...,  2.1350,  2.7325,  0.1760],
         [-0.6037, -0.1550,  0.8888,  ...,  2.1350,  2.7326,  0.1760]],

        [[-0.4055, -0.4849, -0.2018,  ...,  0.6381, -1.3429,  1.1430],
         [ 0.6016,  0.6899,  0.1445,  ...,  0.5265,  1.0655, -0.1205],
         [ 1.1073,  1.7684,  1.8862,  ...,  2.5377,  0.4864,  0.0394],
         ...,
         [ 0.4736, -1.4479,  2.0100,  ...,  2.1350,  2.7324,  0.1760],
         [-0.4182, -1.0957,  1.7640,  ...,  2.1350,  2.7325,  0.1760],
         [-0.6037, -0.1550,  0.8888,  ...,  2.1350,  2.7326,  0.1760]]],
       grad_fn=<AddBackward0>)
torch.Size([2, 50, 512])


In [ ]:
df

,caption,image
0,"[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 1...",1000268201_693b08cb0e.jpg
1,"[1, 16, 17, 18, 1, 19, 20, 15]",1000268201_693b08cb0e.jpg
2,"[1, 21, 16, 7, 18, 1, 19, 22, 15]",1000268201_693b08cb0e.jpg
3,"[1, 21, 16, 7, 23, 11, 24, 25, 22, 15]",1000268201_693b08cb0e.jpg
4,"[1, 21, 16, 3, 1, 4, 5, 17, 18, 1, 19, 26, 15]",1000268201_693b08cb0e.jpg
...,...,...
40163,"[234, 1258, 38, 1, 4025, 3, 561, 35, 1202, 306...",997338199_7343367d7f.jpg
40164,"[1, 75, 3, 1, 4, 161, 112, 1, 164, 202, 15]",997722733_0cb5439472.jpg
40165,"[1, 75, 6, 164, 7, 488, 3, 23, 227, 15]",997722733_0cb5439472.jpg
40166,"[1, 187, 3, 1, 110, 161, 7, 8, 1, 164, 202, 54...",997722733_0cb5439472.jpg


In [ ]:
df['caption'][0]

[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 9, 10, 11, 3, 12, 13, 14, 15]